# ingest.helpers

> helper utilities for ingesting documents

In [ ]:
# | default_exp ingest.helpers

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

from typing import List, Union, Optional
import os
import os.path
import glob
from hashlib import md5
from functools import partial
import datetime
from langchain_core.documents import Document
from onprem.utils import contains_sentence

CAPTION_DELIMITER = '||CAPTION||'
def includes_caption(d:Document):
    """
    Returns True if content of supplied Document includes a table caption
    """
    table_captions = d.metadata.get('table_captions', '')
    if not table_captions: return False
    table_captions = table_captions.split(CAPTION_DELIMITER)
    for c in table_captions:
        if contains_sentence(c, d.page_content):
            return True
    return False


def extract_tables(filepath:Optional[str]=None, docs:Optional[List[Document]]=[]) -> List[Document]:
    """
    Extract tables from PDF and append to end of supplied Document list.
    Accepts either a `filepath` or a list of LangChain `Document` objects 
    all from a **single** file. If `filepath` is empty, the file path of interest 
    is extracted from `docs`. 

    Returns an updated list of Document objects appended with extracted tables.
    """
    from onprem.ingest.pdftables import PDFTables

    # get filepath of document under consideration
    docs = [] if not docs else docs
    if not filepath and not docs:
        raise ValueError('filepath and docs cannot both be empty.')
    if filepath and docs:
        raise ValueError('filepath and docs are mutually exclusive.')
    if docs:
        filepath = None if not docs else docs[0].metadata['source']
    if not filepath: return docs
        
    if extract_extension(filepath) != "pdf": return docs
    pdftab = PDFTables.from_file(filepath, verbose=False)
    md_tables = pdftab.get_markdown_tables()

    # tag document objects that contain extracted tables
    captions = pdftab.get_captions()
    for c in captions:
        for d in docs:
            if contains_sentence(c, d.page_content):
                table_captions = d.metadata.get('table_captions', [])
                if isinstance(table_captions, str):
                    table_captions = table_captions.split(CAPTION_DELIMITER)
                table_captions.append(c)
                d.metadata['table_captions'] = CAPTION_DELIMITER.join(table_captions)

    # augment docs with extracted tables
    tabledocs = []
    for md_table in md_tables:
        tabledoc = Document(page_content=md_table,
                metadata={'source':filepath, 'markdown':True, 'table':True})
        tabledocs.append(tabledoc)
    docs.extend(tabledocs)
    return docs


def extract_files(source_dir:str, follow_links=False, extensions:Optional[Union[dict,list]]=None):
    extensions = list(extensions.keys()) if isinstance(extensions, dict) else extensions
    if os.listdir(source_dir) == []:
        raise ValueError("%s: path is empty" % source_dir)
    walk = os.walk
    for root, _, filenames in walk(source_dir, followlinks=follow_links):
        for filename in filenames:
            if extensions and extract_extension(filename) not in extensions:
                continue
            try:
                yield os.path.join(root, filename)
            except:
                continue

def extract_extension(filepath:str, include_dot=False):
    """
    Extracts file extension (including dot) from file path
    """
    dot = "." if include_dot else ""
    return dot + filepath.rsplit(".", 1)[-1].lower()



def extract_file_dates(filepath):
    """
    Takes a file path and returns an ISO datetime string of last-modified
    and create date of file.

    Returns tuple of the form (create-date, last-modify-date)
    """
    from os.path import getmtime, getctime
    mtimestamp = getmtime(filepath)
    ctimestamp = getctime(filepath)
    mdate = datetime.datetime.fromtimestamp(mtimestamp).isoformat()
    cdate = datetime.datetime.fromtimestamp(ctimestamp).isoformat()
    return cdate, mdate

def iso2date(s):
    return datetime.datetime.fromisoformat(s)


def date2iso(d):
    return d.isoformat()


def md5sum(filepath):
    """
    Perform an MD5 hash of a file.
    """
    with open(filepath, "rb") as fobj:
        data = md5()
        for buf in iter(partial(fobj.read, 128), b""):
            data.update(buf)
    return data.hexdigest()

In [ ]:
#|export

import mimetypes
try:
    import magic

    get_magic = magic._get_magic_type(True)
    MAGIC_INSTALLED = True
except Exception as e:
    get_magic = None
    MAGIC_INSTALLED = False
    warnings.warn(
        "using mimetypes, since magic could not be imported (perhaps try python-magic-bin if on Windows):  %s"
        % (e)
    )

def get_mimetype(filepath):
    with open(filepath, "rb") as fobj:
        buf = fobj.read(4096)
    return get_magic.from_buffer(buf) if isinstance(get_magic, magic.Magic) else ""


def extract_mimetype(filepath):
    """
    Extract mimetype.
    Returns a tuple with extracted mimetype, type, subtype.
    """

    if MAGIC_INSTALLED:
        mtype = get_mimetype(filepath)
    else:
        mtype = mimetypes.guess_type(filepath)[0]
    mtype1 = None
    mtype2 = None
    if isinstance(mtype, str):
        try:
            mtype1, mtype2 = mtype.split("/")
            # Reading from buffer does not work for old office formats
            if "CDFV2" in mtype2 and isinstance(get_magic, magic.Magic):
                mtype = get_magic.from_file(filepath)
                if isinstance(mtype, str):
                    mtype1, mtype2 = mtype.split("/")
        except Exception:
            mtype = None
            mtype1 = None
            mtype2 = None
    return (mtype, mtype1, mtype2)


def is_random_plaintext(extension, mimetype):
    """
    Check mimetype for plain text
    """
    mimetype1, mimetype2 = mimetype.split('/')
    return mimetype1 == "text" or mimetype2 == "xml" or extension in {"txt"}

In [ ]:
# | export



def clean_text(text_s_or_b):
    """
    convert to string and strip.
    """
    text = (
        text_s_or_b.decode(errors="ignore")
        if isinstance(text_s_or_b, bytes)
        else text_s_or_b
    )
    text = text.strip()
    return text

In [ ]:
# | export

METADATA = {'source':None,
            'ocr':False,
            'table' : False,
            'markdown' :False,
            'page' :None,
            'table_captions':None,
            'document_title' : None,
           }

def create_document(page_content:str, 
                    source:str,
                   **kwargs):
    """
    Create document with required metadata keys from `METADATA`.
    """
    kwargs['source'] = source
    metadata = {k: v for k, v in kwargs.items() if k in METADATA}
    return Document(page_content=page_content, 
                    metadata=metadata)

def set_metadata_defaults(docs:List[Document], extra_keys:list=[]):
    """
    Sets Document metadata defaults
    """
    for doc in docs:
        # Keep only keys that are in METADATA or extra_keys
        doc.metadata = {k: v for k, v in doc.metadata.items() if k in METADATA or k in extra_keys}
        # Add keys from METADATA if they don't exist in doc.metadata
        doc.metadata.update({k: v for k, v in METADATA.items() if k not in doc.metadata})
    return docs

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()